In [1]:
import numpy as np
import os

from  PIL import Image
import matplotlib.pyplot as plt
from matplotlib import image
import splitfolders

from tensorflow.python.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, AveragePooling2D, Dropout, Flatten, Dense

from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
import keras

from tensorflow.python.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.callbacks import TensorBoard
import time



input_folder='.\\data_concreate\\D\\'
output_folder='\\data_concreate\\images\\'
InpShape=64

In [ ]:


# # Split with a ratio.
# # To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
# splitfolders.ratio(input_folder, output=output_folder,
#       seed=42, ratio=(.7, .2, .1), group_prefix=None, move=False) # default values



In [ ]:
base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(InpShape, InpShape, 3), # VGG16 expects min 32 x 32
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False


In [2]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      #keras.metrics.CategoricalCrossentropy(name='ccent'),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
   
      
      #keras.metrics.AUC(name='auc'),
     # keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]



In [26]:
inputs = tf.keras.Input(shape=(InpShape,InpShape,3))
x = tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x)
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x)
x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(16,activation='relu')(x)

x = tf.keras.layers.Dense(8,activation='relu')(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)







In [27]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 62, 62, 8)         224       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 31, 31, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 29, 29, 16)        1168      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 14, 14, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 12, 12, 32)        4640

In [28]:

NAME="cement_binary-{}".format(int(time.time())) # log file name for tensorboard


model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(), # default from_logits=False
              metrics=METRICS)



early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1,mode='auto')
learning_rate = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='auto')
tensorboard=TensorBoard(log_dir='./logs/{}'.format(NAME)) ## logs are under logs folder for tensorboard


batch_size = 32 
epochs = 75







In [29]:



train_datagen = ImageDataGenerator(rescale = 1./255
                                  )
training_set = train_datagen.flow_from_directory('./'+output_folder + 'train',
                                                 target_size = (InpShape,InpShape),
                                                 batch_size = batch_size,
                                                 shuffle=True,
                                                 class_mode = 'binary')


Found 12442 images belonging to 2 classes.


In [ ]:
training_set.class_indices

In [30]:
val_datagen = ImageDataGenerator(rescale = 1./255)
val_set = val_datagen.flow_from_directory('./'+output_folder +'val',
                                    
                                            target_size = (InpShape, InpShape),
                                            batch_size = batch_size,
                                            class_mode = 'binary')


Found 3554 images belonging to 2 classes.


In [ ]:
val_set.class_indices

In [31]:
history = model.fit(training_set, validation_data=val_set, 
                     epochs = epochs,callbacks = [learning_rate, early_stop, tensorboard])

Epoch 1/75
389/389 [==============================] - 19s 43ms/step - loss: 0.5584 - accuracy: 0.7438 - precision: 0.7307 - recall: 0.8075 - tp: 6003.0000 - fp: 2212.0000 - tn: 4576.0000 - fn: 1431.0000 - val_loss: 0.4848 - val_accuracy: 0.7983 - val_precision: 0.7936 - val_recall: 0.8299 - val_tp: 1542.0000 - val_fp: 401.0000 - val_tn: 1295.0000 - val_fn: 316.0000 - lr: 0.0010
Epoch 2/75
389/389 [==============================] - 15s 39ms/step - loss: 0.4814 - accuracy: 0.7931 - precision: 0.8073 - recall: 0.7937 - tp: 5162.0000 - fp: 1232.0000 - tn: 4706.0000 - fn: 1342.0000 - val_loss: 0.4379 - val_accuracy: 0.8165 - val_precision: 0.8577 - val_recall: 0.7783 - val_tp: 1446.0000 - val_fp: 240.0000 - val_tn: 1456.0000 - val_fn: 412.0000 - lr: 0.0010
Epoch 3/75
389/389 [==============================] - 15s 39ms/step - loss: 0.4482 - accuracy: 0.8112 - precision: 0.8293 - recall: 0.8044 - tp: 5232.0000 - fp: 1077.0000 - tn: 4861.0000 - fn: 1272.0000 - val_loss: 0.4059 - val_accuracy: 

In [32]:
test_datagen = ImageDataGenerator(rescale = 1./255)


test_set = test_datagen.flow_from_directory('./'+output_folder +'test',
                                            target_size = (InpShape, InpShape),
                                            batch_size = batch_size,
                                            class_mode = 'binary')

Found 1780 images belonging to 2 classes.


In [76]:
#history.model.evaluate(test_datagen)
score = model.evaluate(test_set)

#score = model.evaluate(generator=test_set)
#print('Accuracy:', score[1])
score



56/56 [==============================] - 2s 33ms/step - loss: 0.3687 - accuracy: 0.8612 - precision: 0.8576 - recall: 0.8806 - tp: 819.0000 - fp: 136.0000 - tn: 714.0000 - fn: 111.0000


[0.36867040395736694,
 0.8612359762191772,
 0.8575916290283203,
 0.8806451559066772,
 819.0,
 136.0,
 714.0,
 111.0]

In [78]:
TP=score[4]
FP=score[5]
TN=score[6]
FN=score[7]
ACC=(TP+TN)/(TP+TN+FP+FN)
PRS=TP/(TP+FP)
REC=TP/(TP+FN)
F1=(2*PRS*REC)/(PRS+REC)
print(F1)

TN/(TN+FP)

0.8689655172413793


0.84

Best model results  
56/56 [==============================] - 2s 32ms/step - loss: 0.3017 - accuracy: 0.8685 - precision: 0.8515 - recall: 0.9065 - tp: 843.0000 - fp: 147.0000 - tn: 703.0000 - fn: 87.0000
[0.3016987442970276,  
 0.8685393333435059,  
 0.8515151739120483,  
 0.9064516425132751,  
 843.0,  
 147.0,  
 703.0,  
 87.0]  

inputs = tf.keras.Input(shape=(InpShape,InpShape,3))  
x = tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu')(inputs)  
x = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x)  
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu')(x)  
x = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x)  
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)  
x = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x)  
x = tf.keras.layers.Flatten()(x)  
x = tf.keras.layers.Dropout(0.2)(x)  
x = tf.keras.layers.Dense(16,activation='relu')(x)  

x = tf.keras.layers.Dense(8,activation='relu')(x)  

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)  
model = tf.keras.Model(inputs=inputs, outputs=outputs)  

In [79]:
model_json = model.to_json()
with open("./model/modelbm.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("./model/modelbm.h5")

### Test Model

In [12]:
Modelmk_json = "./model/modelbm.json"
Modelmk_weigths = "./model/modelbm.h5"
from tensorflow.python.keras.models import model_from_json

def get_model(modeljson, weights):
    '''
    Function to load saved model and weights 
    '''
    model_json = open(modeljson, 'r')
    loaded_model_json = model_json.read()
    model_json.close()
    model = model_from_json(loaded_model_json)
    model.load_weights(weights)
    return model



In [13]:
from tensorflow.keras.preprocessing import image
def model_predict(img: image, model, dima: int, dimb: int):
    '''
    Get the image data and return predictions
    '''
    img = img.resize((dima, dimb))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x/255
    preds = model.predict(x)

    return preds

In [16]:

a = '021-100.jpg'
im=Image.open(a)



modelmk = get_model(Modelmk_json, Modelmk_weigths)

        # Make predictions
predsmk = model_predict(im, modelmk, InpShape, InpShape)[0][0]
#predsmk =predsmk *100
pred=predsmk *100
pred=pred.round(1)
pred


0.7